#PCOS DETECTION USING YOLOV5 MODEL


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/zippedfile.zip -d /content/PCOS/

In [3]:
import os
import random
import shutil
# Set paths
dataset_path = "/content/PCOS/train"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

# Output directories
train_images = os.path.join(dataset_path, "train", "images")
train_labels = os.path.join(dataset_path, "train", "labels")
val_images = os.path.join(dataset_path, "val", "images")
val_labels = os.path.join(dataset_path, "val", "labels")
test_images = os.path.join(dataset_path, "test", "images")
test_labels = os.path.join(dataset_path, "test", "labels")

# Create directories
for path in [train_images, train_labels, val_images, val_labels, test_images, test_labels]:
    os.makedirs(path, exist_ok=True)

# List all image files
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Shuffle and split
random.seed(42)
random.shuffle(image_files)
total = len(image_files)
train_end = int(0.7 * total)
val_end = int(0.85 * total)
train_files = image_files[:train_end]
val_files = image_files[train_end:val_end]
test_files = image_files[val_end:]

# Helper to move files
def move_files(file_list, src_img_dir, src_lbl_dir, dest_img_dir, dest_lbl_dir):
    for file in file_list:
        img_path = os.path.join(src_img_dir, file)
        label_path = os.path.join(src_lbl_dir, file.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))

        if os.path.exists(img_path):
            shutil.move(img_path, os.path.join(dest_img_dir, file))
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_lbl_dir, os.path.basename(label_path)))
# Move files accordingly
move_files(train_files, images_path, labels_path, train_images, train_labels)
move_files(val_files, images_path, labels_path, val_images, val_labels)
move_files(test_files, images_path, labels_path, test_images, test_labels)

print(f"✅ Dataset split done: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")


✅ Dataset split done: 2468 train, 529 val, 529 test


In [6]:
import os

image_dir = "/content/PCOS/train/val/images"  # Update with your image folder
label_dir = "/content/PCOS/train/val/labels"

image_exts = (".jpg", ".png", ".jpeg")  # Add other formats if needed
label_ext = ".txt"  # YOLO format uses .txt files for annotations

# Get filenames (without extensions)
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith(image_exts)}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith(label_ext)}

# Find mismatches
missing_labels = image_files - label_files
extra_labels = label_files - image_files

print(f"✅ Matching Files: {len(image_files & label_files)}")
print(f"⚠ Missing Label Files: {len(missing_labels)}")
print(f"⚠ Extra Label Files: {len(extra_labels)}")

if missing_labels:
    print("\n🛑 Images without labels:", missing_labels)
if extra_labels:
    print("\n🛑 Labels without images:", extra_labels)


✅ Matching Files: 529
⚠ Missing Label Files: 0
⚠ Extra Label Files: 0


In [7]:
data_yaml = """
train: /content/PCOS/train/train
val: /content/PCOS/train/val
test: /content/PCOS/train/test

nc: 1  # Only one class (PCOS)
names: ['PCOS']
"""

# Save the file
with open("/content/PCOS/train/data.yaml", "w") as f:
    f.write(data_yaml)

print("data.yaml file created successfully!")


data.yaml file created successfully!


In [15]:
import shutil

folder_path = "/content/PCOS"
output_zip = "/content/PCOS.zip"

shutil.make_archive(output_zip.replace('.zip', ''), 'zip', folder_path)
print("Folder zipped successfully!")
from google.colab import files

files.download(output_zip)

Folder zipped successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17395, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 17395 (delta 57), reused 20 (delta 20), pack-reused 17320 (from 3)
Receiving objects: 100% (17395/17395), 16.26 MiB | 28.96 MiB/s, done.
Resolving deltas: 100% (11926/11926), done.


In [9]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

In [11]:
!python train.py --img 640 --batch 16 --epochs 50 --data /content/PCOS/train/data.yaml --weights yolov5s.pt --device 0

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      34/49      4.75G    0.02149    0.02455          0         97        640:  16% 25/155 [00:09<00:50,  2.58it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      34/49      4.75G    0.02145    0.02425          0         51        640:  17% 26/155 [00:10<00:48,  2.65it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      34/49      4.75G    0.02154      0.024          0         62        640:  17% 27/155 [00:10<00:47,  2.70it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp

In [12]:
!python val.py --data /content/PCOS/train/data.yaml --weights runs/train/exp/weights/best.pt --img 640 --task test


val: data=/content/PCOS/train/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-413-gf2f86eb3 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/PCOS/train/test/labels... 529 images, 182 backgrounds, 0 corrupt: 100% 529/529 [00:00<00:00, 806.16it/s]
test: New cache created: /content/PCOS/train/test/labels.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 17/17 [00:08<00:00,  2.04it/s]
                   all        529       1850      0.795      0.914      0.942      0.699
Speed: 0.9ms pre-process, 6.7ms inference,

In [24]:
from google.colab import files
files.download('/content/yolov5/runs/train/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# testing on the image


In [ ]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
!pip install gradio


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


In [26]:
import sys
sys.path.append('/content/yolov5')

In [27]:
# Step 1: Install and import dependencies


import gradio as gr
import torch
import numpy as np
from PIL import Image
import cv2
import sys


from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.plots import Annotator
from utils.augmentations import letterbox

# Load trained model
device = torch.device('cpu')
model = DetectMultiBackend('/content/best2.pt', device=device)

stride = model.stride
names = model.names
model.eval()

# Step 3: Define detection function
def detect_pcos(image: Image.Image):
    try:
        # Convert image to NumPy array and BGR format
        img_np = np.array(image)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        # Resize with padding
        img_resized = letterbox(img_bgr, stride=stride, auto=True)[0]
        img_resized = img_resized.transpose((2, 0, 1))
        img_resized = np.ascontiguousarray(img_resized)
        # Convert to torch tensor
        im_tensor = torch.from_numpy(img_resized).to(model.device).float()
        im_tensor /= 255.0  # Normalize
        if im_tensor.ndimension() == 3:
            im_tensor = im_tensor.unsqueeze(0)  # Add batch dim
        # Inference
        pred = model(im_tensor)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)
        # Annotate image
        img_display = img_bgr.copy()
        annotator = Annotator(img_display, line_width=2)
        count = 0
        for det in pred:
            if det is not None and len(det):
                count = len(det)
                det[:, :4] = scale_boxes(im_tensor.shape[2:], det[:, :4], img_display.shape).round()
                for *xyxy, conf, cls in det:
                    label = f'{names[int(cls)]} {conf:.2f}'
                    annotator.box_label(xyxy, label, color=(0, 0, 255))

        output_img = annotator.result()
        output_pil = Image.fromarray(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
        result_text = f"✅ PCOS Detected (Follicles: {count})" if count >= 4 else f"❌ No PCOS Detected (Follicles: {count})"
        return output_pil, result_text

    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Step 4: Create Gradio interface
interface = gr.Interface(
    fn=detect_pcos,
    inputs=gr.Image(type="pil", label="Upload Ultrasound Image"),
    outputs=[gr.Image(type="pil", label="Detected Image"), gr.Textbox(label="Detection Result")],
    title="PCOS Detection from Ultrasound",
    description="Upload an ultrasound image. This tool uses YOLOv5 to detect follicles and determine PCOS."
)

interface.launch(share=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be5707013009c39dcf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
! pip install pytesseract
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [7]:
import pytesseract
from PIL import Image

def detect_pcos(image: Image.Image):
    try:
        # Step 1: Resize image to 640x640
        image = image.resize((640, 640))

        # Step 2: Convert to grayscale for text detection
        gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

        # Step 3: Text detection using OCR
        text_detected = pytesseract.image_to_string(gray).strip()
        if len(text_detected) > 5:
            return None, "❌ Text detected in image. Please upload a clean ultrasound scan without text."

        # Step 4: Convert image to NumPy and BGR format for YOLOv5
        img_np = np.array(image)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        # Step 5: Resize with letterbox padding
        img_resized = letterbox(img_bgr, stride=stride, auto=True)[0]
        img_resized = img_resized.transpose((2, 0, 1))
        img_resized = np.ascontiguousarray(img_resized)

        # Step 6: Convert to torch tensor
        im_tensor = torch.from_numpy(img_resized).to(model.device).float()
        im_tensor /= 255.0
        if im_tensor.ndimension() == 3:
            im_tensor = im_tensor.unsqueeze(0)

        # Step 7: Inference
        pred = model(im_tensor)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

        # Step 8: Annotate image
        img_display = img_bgr.copy()
        annotator = Annotator(img_display, line_width=2)
        count = 0

        for det in pred:
            if det is not None and len(det):
                count = len(det)
                det[:, :4] = scale_boxes(im_tensor.shape[2:], det[:, :4], img_display.shape).round()
                for *xyxy, conf, cls in det:
                    label = f'{names[int(cls)]} {conf:.2f}'
                    annotator.box_label(xyxy, label, color=(0, 0, 255))

        output_img = annotator.result()
        output_pil = Image.fromarray(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
        result_text = f"✅ PCOS Detected (Follicles: {count})" if count >= 4 else f"❌ No PCOS Detected (Follicles: {count})"

        return output_pil, result_text

    except Exception as e:
        return None, f"❌ Error: {str(e)}"


In [ ]:
!pip install gradio

In [13]:


interface = gr.Interface(
    fn=detect_pcos,
    inputs=gr.Image(type="pil", label="Upload Ultrasound Image"),
    outputs=[
        gr.Image(type="pil", label="Detected Image"),
        gr.Textbox(label="Detection Result")
    ],
    title="PCOS Detection from Ultrasound",
    description="Upload an ultrasound image (640x640). This tool detects PCOS based on follicle count using YOLOv5."
)

interface.launch(share=True)  # Use share=False if you want local-only


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a0b98d5fc612c603ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
